AOC [23.05](https://adventofcode.com/2023/day/5)

You're asked to help with a food production problem. Their almanac lsts all the seeds to be planted, what type of fertilizer to be used, how much water, etc.

Almanac lists seeds to be planted, then maps to convert these numbers to soil types.

maps have three numbers: dest range start, source range start, range length:
50 98 2 --> 50,51 98,99 
52 50 48 --> 52-97 50-95 

source numbers not mapped have same destination number

Solution:
1. I think I can solve this by setting up dictionaries for each of the different maps: {98:50,99:51,50:52,etc}
1. Then I can just put in the seed number and find the corresponding value; items not in the dictionary keep their same number
1. parsing the input data may be more difficult than the actual mapping process!
* read a line. If not numbers, start a new dictionary
* read next lines until a blank. put these values into the dictionary



In [ ]:
testData = '''
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4'''

def getSeeds(line):
    head,_,tail = line.partition(':')
    return [int(x) for x in tail.split()]

import re
findWords = re.compile(r'(seeds)|(map)')

for line in testData.split('\n'):
    if re.search(findWords,line): # text or numbers?
        if 'seeds' in line:
            seedNums = getSeeds(line)
        else:
            head,_,tail = line.partition('-')
            currDict = f"{head}=\{\}"
            exec(currDict) # create a dictionary (seed={},soil={},fertilizer={}
            currDict = head
    else: # if not text, the line is either nubmbers or spaces
        if line<>'': # ignore blank lines
            destStart,sourceStart,length = line.split() # split on blanks and assign to variables
            for num in range(sourceStart,length+1):
                currDict[num] = destStart + num     #fill the dictionary with source and destination data

dictionaries = ['seed','soil','fertilizer','water','light','temperature','humidity']
lowestNum = 10**10

for number in seedNums:
    for dd in dictionaries:
        if number in dd:
            number = dd[number]
    if number < lowestNum:
        lowestNum = number

print(f"lowest number is {lowestNum}")

    

seeds: 79 14 55 13
seed-to-soil map:
soil-to-fertilizer map:
fertilizer-to-water map:
water-to-light map:
light-to-temperature map:
temperature-to-humidity map:
humidity-to-location map:


OK, I can't figure out a way to do this with the dictionaries as I had hoped to. So let's try something else.

I've looked at some solutions on Reddit. Found [this one](https://github.com/torbensky/advent-of-code-2023/blob/main/day05/solution.py) useful for reading in the data. Won't look at the rest of the code (yet).

1. Get seed data
1. Break the data into clumps by splitting on \n\n (since there's a blank line b/t groups of data)
1. read each clump into a "map" list of tuples [(target, source, range),(target,source,range)...]
1. Take each seed number and run it through the different mapping functions


In [47]:
clumps = testData.split('\n\n')

clumps[1].split('\n')


['seed-to-soil map:', '50 98 2', '52 50 48']

In [18]:
clumps[0]

'\nseeds: 79 14 55 13'

In [20]:
head,_,tail = clumps[0].partition(':')
seedNums = [int(x) for x in tail.split()]
seedNums

[79, 14, 55, 13]

NameError: name 'makeMap' is not defined

In [28]:
clumps[2].splitlines()

['soil-to-fertilizer map:', '0 15 37', '37 52 2', '39 0 15']

In [35]:
md = clumps[2].splitlines()
md[0].split()[0].split('-')[0]


'soil'

In [36]:
md[0].split()[0].split('-')[2]

'fertilizer'

In [42]:
[list(map(int,md[1].split())) for x in md[1:]]

[[0, 15, 37], [0, 15, 37], [0, 15, 37]]

In [49]:
def makeMap(data):
    map_data = data.splitlines()
    sourceMap = map_data[0].split()[0].split('-')[0]
    destMap =   map_data[0].split()[0].split('-')[2]
    translation = [list(map(int, x.split())) for x in map_data[1:]]
    return (sourceMap, destMap, translation)

mappers = {makeMap(mapper)[0]: makeMap(mapper)[1:] for mapper in clumps[1:]}
mappers


{'seed': ('soil', [[50, 98, 2], [52, 50, 48]]),
 'soil': ('fertilizer', [[0, 15, 37], [37, 52, 2], [39, 0, 15]]),
 'fertilizer': ('water', [[49, 53, 8], [0, 11, 42], [42, 0, 7], [57, 7, 4]]),
 'water': ('light', [[88, 18, 7], [18, 25, 70]]),
 'light': ('temperature', [[45, 77, 23], [81, 45, 19], [68, 64, 13]]),
 'temperature': ('humidity', [[0, 69, 1], [1, 0, 69]]),
 'humidity': ('location', [[60, 56, 37], [56, 93, 4]])}

In [58]:
mappers['seed'][1]

[[50, 98, 2], [52, 50, 48]]

OK, that works, but it's more complex than I need. Since I'm applying the maps in sequence, I don't need to know their names -- I'm just going to apply them 1->2->3->...

So let's just make a list of the various maps.

In [61]:
def makeMap(data): # input clump of data, return list of map
    return [list(map(int,x.split())) for x in data.splitlines()[1:]] # skip line of text

maplist = [makeMap(x) for x in clumps[1:]]

maplist

[[[50, 98, 2], [52, 50, 48]],
 [[0, 15, 37], [37, 52, 2], [39, 0, 15]],
 [[49, 53, 8], [0, 11, 42], [42, 0, 7], [57, 7, 4]],
 [[88, 18, 7], [18, 25, 70]],
 [[45, 77, 23], [81, 45, 19], [68, 64, 13]],
 [[0, 69, 1], [1, 0, 69]],
 [[60, 56, 37], [56, 93, 4]]]

OK, so now I've got a list of maps. I can cycle through them and do the seed number matching...

In [87]:
def applyMap(seedN,mapping):
    for m in mapping: 
        if m[1] <= seedN <= m[1] + m[2]: # if seedN is in the source range
            #print(f"found in map {m}")
            return seedN - m[1] + m[0]
    return seedN # not found in any range? return original value

minN = 10**8

for s in seedNums:
    #print(f"orig seed number {s}")
    for mapping in maplist:
        s = applyMap(s,mapping)
    if s < minN:
        minN = s

print(minN)


    

35


OK, test data works. Re-write to get everything into one cell and run on actual data

In [94]:

def makeMap(data): # input clump of data, return list of map
    return [list(map(int,x.split())) for x in data.splitlines()[1:]] # skip line of text

def applyMap(seedN,mapping):
    for m in mapping: 
        if m[1] <= seedN <= m[1] + m[2]: # if seedN is in the source range
            #print(f"found in map {m}")
            return seedN - m[1] + m[0]
    return seedN # not found in any range? return original value

#group = testData.split('\n\n') # get the data; it will be in groups of data

with open ('2305input.txt') as f_in:
    groups = f_in.read().split('\n\n')

head,_,tail = groups[0].partition(':')
seedNums = [int(x) for x in tail.split()]

maplist = [makeMap(x) for x in groups[1:]] # ma

minN = 10**12

for s in seedNums:
    #print(f"orig seed number {s}")
    for mapping in maplist:
        s = applyMap(s,mapping)
    if s < minN:
        minN = s

print(minN)


346433842


That's correct.

Part2:
Initial input isn't four seed nubmers [$79 14 55 13$], it's a range (beg,length) [$(79,14),(55,13)$]. So 79 -> 92, 55 -> 67.

What is the lowest value for all these?

Solution:
1. could try brute force: read the pairs and make a much longer list of inputs. This would work on the test data, but given the size of the numbers in the actual data, this will take forever (see next cell)
1. test for beg and end of range before testing (if beg and beg+end not in the map, go to the next map)
1. Even this is not great: the first range is >500m


In [103]:
for i in range(0,len(seedNums)//2,2):
    print([(seedNums[0+i],seedNums[0+i+1])])

[(919339981, 562444630)]
[(3366006921, 67827214)]
[(1496677366, 101156779)]
[(4140591657, 5858311)]
[(2566406753, 71724353)]


In [106]:
head,_,tail = groups[0].partition(':')
sNs = tail.split()
seedRanges = [(sNs[0+i],sNs[1+i+1]) for i in range(0,len(sNs)//2,2) ]
seedRanges

[('919339981', '3366006921'),
 ('3366006921', '1496677366'),
 ('1496677366', '4140591657'),
 ('4140591657', '2566406753'),
 ('2566406753', '2721360939')]

In [109]:
def makeMap(data): # input clump of data, return list of map
    return [list(map(int,x.split())) for x in data.splitlines()[1:]] # skip line of text

def applyMap(sp,mapping):
    for seedN in range(sp[0],sp[0]+sp[1]):
        for m in mapping: 
            if m[1] <= seedN <= m[1] + m[2]: # if seedN is in the source range
                #print(f"found in map {m}")
                return seedN - m[1] + m[0]
        return seedN # not found in any range? return original value

groups = testData.split('\n\n') # get the data; it will be in groups of data

#with open ('2305input.txt') as f_in:
#    groups = f_in.read().split('\n\n')

head,_,tail = groups[0].partition(':')
sNs = tail.split()
seedRanges = [(int(sNs[0+i]),int(sNs[1+i+1])) for i in range(0,len(sNs)//2,2) ]

maplist = [makeMap(x) for x in groups[1:]] # ma

minN = 10**12

for sPair in seedRanges:
    #print(f"orig seed number {s}")
    for mapping in maplist:
        s = applyMap(sPair,mapping)
    if s < minN:
        minN = s

print(minN)


83
